# Cvičenie 4: Pohľad na tabuľku

Na dnešnom cvičení si precvičíme prácu so súbormi a základnými údajovými štruktúrami v Pythone. Cieľom je poukázať na typické prípady použitia týchto údajových štruktúr ako aj na postup pri spracovaní údajov a možné rozdelenie úloh pre štruktúrované riešenia.

Ukážkovým príkladom bude spracovanie zoznamu futbalových výsledkov a generovanie tabuľky stavu ligy na základe týchto výsledkov. [Na tomto odkaze](sources/lab04/buli_results.txt) nájdete súbor s názvom `buli_results.txt`, ktorý obsahuje výsledky zápasov z prvých 23 kôl nemeckej Bundesligy. Každý riadok reprezentuje jeden zápas a má formu:

`meno_domáceho_tímu počet_gólov_domácich - počet_gólov_hostí meno_hosťujúceho_tímu`.

Ako môžete vidieť v súbore, väčšina tímov má meno, ktoré sa skladá z viacerých slov, alebo obsahuje aj čísla, napríklad `1. FC Köln`. Ako platí všeobecne vo futbale, tím, ktorý strelí viac gólov, zápas vyhrá a získa 3 body. Ak oba tímy strelia rovnaký počet gólov (alebo ani jeden z nich gól nestrelí), oba odchádzajú zo zápasu s 1 bodom. Za prehru tímy bod nedostanú. 

Vaším cieľom na dnešnom cvičení je zo súboru vygenerovať tabuľku, ktorá bude mať približne nasledovnú formu:

```
|Rank|Team                    | G| W| D| L| GF| GA| P|
------------------------------------------------------
|   1|FC Bayern München       |23|14| 7| 2| 66| 22|49|
|   2|Borussia Dortmund       |23|16| 1| 6| 47| 28|49|
|   3|1. FC Union Berlin      |23|13| 5| 5| 35| 27|44|
|   4|RB Leipzig              |23|12| 6| 5| 46| 29|42|
|   5|SC Freiburg             |23|12| 6| 5| 35| 32|42|
|   6|Eintracht Frankfurt     |22|11| 5| 6| 43| 31|38|
|   7|1. FSV Mainz 05         |23|10| 5| 8| 38| 34|35|
|   8|VfL Wolfsburg           |22| 9| 6| 7| 40| 29|33|
|   9|Borussia Mönchengladbach|23| 8| 6| 9| 38| 39|30|
|  10|SV Werder Bremen        |23| 9| 3|11| 35| 43|30|
|  11|Bayer 04 Leverkusen     |22| 8| 4|10| 36| 36|28|
|  12|1. FC Köln              |23| 6| 9| 8| 32| 36|27|
|  13|FC Augsburg             |23| 8| 3|12| 28| 39|27|
|  14|Hertha BSC              |22| 5| 5|12| 27| 40|20|
|  15|VfB Stuttgart           |23| 4| 7|12| 28| 40|19|
|  16|TSG 1899 Hoffenheim     |23| 5| 4|14| 29| 42|19|
|  17|FC Schalke 04           |23| 4| 7|12| 18| 42|19|
|  18|VfL Bochum              |23| 6| 1|16| 24| 56|19|
```

[Kostru riešenia nájdete tu](sources/lab04/lab04.py), alebo môžete pracovať priamo v tomto Jupyter notebooku.

## 1. Načítanie zo súboru

Prvou úlohou je načítať údaje zo súboru s výsledkami. Tieto výsledky zatiaľ nijak nespracujeme, iba ich načítame pre ďalšie spracovanie.

**Úloha:** Implementujte funkciu `load_results`, ktorá načíta obsah súboru a vráti zoznam reťazcov, kde jeden reťazec reprezentuje jeden riadok zo súboru. Cestu k súboru dostane funkcia ako parameter `file_path`.

In [133]:
import pandas as pd

In [134]:
def load_results(file_path):
    file = open(file_path, encoding="utf8").read()
    data = []
    for ind, line in enumerate(file.split('\n')):
        data.append(line)
    return data


## 2. Načítanie výsledkov

Funkciu pre načítanie výsledkov zo súboru už máme hotovú, avšak všetky informácie máme v jednom reťazci, čo nám veľmi nepomôže. Pri vytvorení tabuľky potrebujeme s týmito údajmi narábať rôznymi spôsobmi, a práve preto je potrebné, aby sme z reťazca reprezentujúceho jeden riadok zo súboru načítali pre nás dôležité dáta.

Konkrétne sú to mená dvoch tímov, a počet nimi strelených gólov. Nezabudnite na to, že každý riadok obsahuje tieto informácie v poradí `domáci_tím domáce_góly - hostia_góly hosťujúci_tím`. Mená tímov sú ale komplikované, obsahujú viacero slov, aj čísla, takže pri načítaní týchto údajov musíme byť opatrní.

**Úloha:** Implementujte funkciu `load_score`, ktorá má jeden parameter - `line`, ktorý je reťazec: riadok načítaný zo súboru. Funkcia vráti štvoricu údajov:
* meno domáceho tímu: str
* počet gólov strelených domácim tímom: int
* meno hosťujúceho tímu: str
* počet gólov hostí: int

In [135]:
def load_score(line):
    splitedInMiddle = line.split(' - ');
    goalsHome = int(splitedInMiddle[0][-1]);
    goalsAway = int(splitedInMiddle[1][0]);
    homeName = splitedInMiddle[0][:-2];
    awayName = splitedInMiddle[1][2:];
    
    return (homeName, goalsHome, awayName, goalsAway);

    

## 3. Zoznam tímov

Pred tým než sa spustíme do vytvorenia tabuľky, potrebujeme zistiť, koľko tímov máme v lige a ktoré sú to. Zatiaľ máme informácie o všetkých zápasoch, avšak jednotlivé tímy sa nachádzajú v týchto záznamoch viackrát.

**Úloha:** Implementujte funkciu `get_team_list`, ktorá vráti zoznam mien tímov v lige. Ako parameter dostane zoznam štvoríc `scores`, kde jedna štvorica je výstupom funkcie `load_score`, teda reprezentuje údaje o jednom zápase. Ak ste šikovní, funkciu napíšete na jeden riadok.

In [136]:
def get_team_list(scores):
    return list(set([i[0] for i in scores] + [i[2] for i in scores]));



## 4. Vytvorenie tabuľky

Už vieme, ako načítať údaje zo súboru, tieto údaje vieme predspracovať a prekonvertovať do potrebnej podoby, a máme už aj zoznam tímov v lige. Nezostáva nám nič iné len vygenerovať zatiaľ prázdnu tabuľku, ktorú postupne budeme aktualizovať na základe výsledkov zápasov.

Tabuľku samozrejme vieme reprezentovať rôznymi spôsobmi, dnes si zvolíme spôsob, kde tabuľka je reprezentovaná ako zoznam máp (dictionary). Teda pre každý tím vytvoríme dictionary, ktorý bude obsahovať informácie o výkone tímu v sezóne. Každý dictionary by mal obsahovať nasledovné údaje:
* meno tímu (`Team`)
* počet odohraných zápasov (`G`)
* počet výhier (`W`)
* počet remíz (`D`)
* počet prehier (`L`)
* počet strelených gólov (`GF`)
* počet inkasovaných gólov (`GA`)
* počet získaných bodov (`P`).

**Úloha:** Implementujte funkciu `create_table`, ktorá vygeneruje prázdnu tabuľku pre ligu na základe zoznamu mien tímov v lige `team_names`. Funkcia vráti zoznam máp, pričom mapy obsahujú zatiaľ iba mená tímov. Pre ostatné údaje uložené v mapách zvoľte vhodné začiatočné hodnoty.

In [137]:
def create_table(team_names):
    return [{'Team': x, 'G': 0, 'W': 0, 'D': 0, 'L': 0, 'GF': 0, 'GA': 0, 'P': 0} for x in team_names]



## 5. Vyhľadávanie v tabuľke

Už sa čoskoro dostaneme k spracovaniu výsledkov a k naplneniu tabuľky užitočnými údajmi, pred tým ale potrebujeme implementovať dve pomocné funkcie.

Prvá z nich rieši vyhľadávanie v tabuľke podľa mena tímu a vráti mapu (dictionary) pre daný tím. Ak záznam o príslušnom tíme nenájde, funkcia vráti hodnotu `None`.

**Úloha:** Implementujte funkciu `get_team_row`, ktorá nájde záznam pre tím `team_name` v tabuľke `table`. Ak takýto záznam neexistuje, funkcia vráti `None`.

In [138]:
def get_team_row(table, team_name):
    # finds the dictionary corresponding to the team in a table
    return [x for x in table if x['Team'] == team_name][0] if len([x for x in table if x['Team'] == team_name]) else None
        


## 6. Výpočet bodov

Druhá pomocná funkcia bude mať na starosti výpočet bodov získaných tímom tak, že prechádza tabuľkou a počet bodov vyráta pre každý tím. Nezabudnite, že za výhru tím získa 3 body a za remízu 1.

**Úloha:** Implementujte funkciu `update_points`, ktorá aktualizuje počet získaných bodov pre každý tím v tabuľke `table`. Funkcia nemá návratovú hodnotu, upravuje priamo tabuľku.

In [139]:
def update_points(table):
    for team in table:
        team['P'] = team['W'] * 3 + team['D']
        

## 7. Aktualizácia tabuľky

Teraz už nám nič nebráni v naplnení tabuľky reálnymi údajmi. Avšak spustiť sa do spracovania všetkých výsledkov naraz by bolo dosť nerozumné, takže v prvom rade vytvoríme funkciu, ktorá spracuje výsledok jedného zápasu a aktualizuje tabuľku.

**Úloha:** Implementujte funkciu `add_match_to_table`, ktorá spracuje výsledok jedného zápasu a aktualizuje tabuľku podľa výsledku. Funkcia má dva parametre: `table` je zoznam reprezentujúci tabuľku, a `match_info` je štvorica hodnôt ako návratová hodnota z funkcie `load_score` (krok 2). Funkcia zoberie informácie o zápase a na základe nich aktualizuje údaje v tabuľke pre dva tímy. Návratovú hodnotu nemá, upravuje priamo tabuľku.

In [140]:
def add_match_to_table(table, match_info):
    # updates table based on a match result
    # input is a list representing the table and
    # a tuple representing the match result
    # updates table directly, returns nothing
    homeTeam = get_team_row(table, match_info[0])
    awayTeam = get_team_row(table, match_info[2])
    if (match_info[1] > match_info[3]):
        homeTeam['W'] += 1
        awayTeam['L'] += 1
    elif (match_info[1] < match_info[3]):
        homeTeam['L'] += 1
        awayTeam['W'] += 1
    else:
        homeTeam['D'] +=1;
        awayTeam['D'] +=1;
    
    for team in table:
        if team['Team'] == homeTeam['Team']:
            team['W'] = homeTeam['W']
            team['L'] = homeTeam['L']
            team['D'] = homeTeam['D']
            team['GF'] += match_info[1]
            team['GA'] += match_info[3]
            team['G'] += 1;
        elif team['Team'] == awayTeam['Team']:
            team['W'] = awayTeam['W']
            team['L'] = awayTeam['L']
            team['D'] = awayTeam['D']
            team['GF'] += match_info[3]
            team['GA'] += match_info[1]
            team['G'] += 1;
            
        

## 8. Naplnenie tabuľky údajmi

A teraz konečne dozrel čas pre našu megafunkciu, ktorá zoberie výsledky uložené v súbore a z nich vygeneruje tabuľku. Samozrejme pri tom využijeme už implementované funkcie z bodov 1-7.

**Úloha:** Implementujte funkciu `generate_table`, ktorá má jeden parameter: cestu k súboru s výsledkami futbalových zápasov (`results_file_path`). Funkcia načíta údaje z tohto súboru, vygeneruje tabuľku a následne túto tabuľku naplní údajmi podľa výsledkov. Funkcia vracia tabuľku, teda zoznam máp.

In [141]:
def generate_table(results_file_path):
    # gets path to file with results
    # generates table and fills it with values based on match results
    # returns the table as a list of dictionaries
    scores = [load_score(i) for i in load_results(results_file_path)];
    table = create_table(get_team_list(scores));
    for score in scores:
        add_match_to_table(table, score);
    update_points(table);
    return table;
        
generate_table('sources/lab04/buli_results.txt');

## 9. Výpis tabuľky

Vaša implementácia funkcie `generate_table` je bez pochýb skvelá, ale výstup z nej nie je príliš prehľadná. Práve preto vytvoríme ďalšiu funkciu, ktorá zoberie tabuľku s dátami a vypíše ju vo forme, ktorá je viac prehľadná pre človeka.

**Úloha:** Implementujte funkciu `print_table`, ktorá dostane ako parameter (`table`) tabuľku, teda zoznam máp a vypíše ju na štandardný výstup ako prehľadnú tabuľku. Pri generovaní výstupu sa môžete inšpirovať tabuľkou z úvodu cvičenia.

In [142]:
def print_table(table):
    display(pd.DataFrame.from_dict(table, orient='columns'));
    
print_table(generate_table('sources/lab04/buli_results.txt'));

,Team,G,W,D,L,GF,GA,P
0,1. FC Köln,23,6,9,8,32,36,27
1,Borussia Dortmund,23,16,1,6,47,28,49
2,TSG 1899 Hoffenheim,23,5,4,14,29,42,19
3,Bayer 04 Leverkusen,23,9,4,10,40,37,31
4,Eintracht Frankfurt,23,11,6,6,45,33,39
5,SV Werder Bremen,23,9,3,11,35,43,30
6,RB Leipzig,23,12,6,5,46,29,42
7,FC Schalke 04,23,4,7,12,18,42,19
8,1. FSV Mainz 05,23,10,5,8,38,34,35
9,FC Bayern München,23,14,7,2,66,22,49


## 10. Zoradenie tabuľky

Ak vaše riešenie vyskúšate viackrát, tak si všimnete, že poradie tímov v tabuľke je viac-menej náhodná (závisí od poradia pridania tímov do tabuľky). Práve preto implementujeme ešte jednu funkciu, ktorá tímy zoradí podľa podmienky. V tabuľkách sú tímy zoradené podľa počtu získaných bodov, gólového rozdielu (počet strelených - počet inkasovaných) a počtu vyhraných zápasov.

**Úloha:** Implementujte funkciu `sort_table`, ktorá zoradí tímy v tabuľke podľa pravidiel určovania poradia; využite pri tom existujúce funkcie Pythonu. Funkcia vracia zoradenú tabuľku a má jeden parameter - tabuľku, ktorú potrebuje zoradiť. Poradie tímov určte podľa nasledovných pravidiel:
1. počet získaných bodov;
2. gólový rozdiel (počet strelených gólov - počet inkasovaných gólov)
3. počet vyhraných zápasov.

**Poznámka:** Pre naše účely to potrebné nebude, ale existujú aj ďalšie pravidlá pre určenie poradia tímov, ak majú horné tri ukazovatele úplne rovnaké. Môžete si ich naštudovať a tak rozšíriť vaše riešenie.

In [143]:
def sort_table(table):
    return sorted(table, key=lambda x: (x['P'], (x['GF'] - x['GA'], x['W'])), reverse=True)

## 11. Spojíme to celé

Keď už máme všetky funkcie implementované, môžeme ich spojiť a tak vyskúšať fungovanie nášho programu. Postupným zavolaním posledných troch funkcií dokážeme zo súboru s výsledkami vypísať prehľadnú tabuľku. Naše riešenie je dokonca všeobecné, t.j. môžete ho využiť pre ľubovoľný súbor až kým forma zápisu výsledkov je dodržaná.

In [144]:
table = generate_table("sources/lab04/buli_results.txt")

table = sort_table(table)

print_table(table)

,Team,G,W,D,L,GF,GA,P
0,FC Bayern München,23,14,7,2,66,22,49
1,Borussia Dortmund,23,16,1,6,47,28,49
2,1. FC Union Berlin,23,13,5,5,35,27,44
3,RB Leipzig,23,12,6,5,46,29,42
4,SC Freiburg,23,12,6,5,35,32,42
5,Eintracht Frankfurt,23,11,6,6,45,33,39
6,1. FSV Mainz 05,23,10,5,8,38,34,35
7,VfL Wolfsburg,23,9,7,7,42,31,34
8,Bayer 04 Leverkusen,23,9,4,10,40,37,31
9,Borussia Mönchengladbach,23,8,6,9,38,39,30


Vašu implementáciu môžete otestovať aj na výsledkoch z anglickej Premier League (súbor [`pl_results.txt`](sources/lab04/pl_results.txt)), španielskej La Ligy ([`laliga_results.txt`](sources/lab04/laliga_results.txt)), z talianskej Seria A ([`seriea_results.txt`](sources/lab04/seriea_results.txt)), alebo môžete si vytvoriť vlastný súbor s výsledkami pri dodržaní formátu.

## Doplňujúce úlohy

1. Rozšírte tabuľku o jeden stĺpec s gólovým rozdielom (goal difference `GD = GF - GA`).
2. Upravte funkciu `sort_table` tak, aby tabuľka sa dala zoradiť podľa vybraných parametrov.
3. Pridajte funkciu pre ukladanie tabuľky do súboru vo [formáte csv](https://en.wikipedia.org/wiki/Comma-separated_values).
4. V niektorých športoch ako napríklad v hokeji zápasy nemôžu končiť remízou. Pri remíze po riadnej hracej dobe zápas pokračuje predĺžením, takýto výsledok zápasu môže byť zapísaný napríklad `domáci 2e - 1 hostia`, kde `e` reprezentuje výhru po predĺžení. Navrhnite a implementujte spôsob spracovania takýchto výsledkov. Ako ukážkové vstupy môžete použiť výsledky zápasov zo slovenskej hokejovej Extraligy (súbor [`extraliga_results.txt`](sources/lab04/extraliga_results.txt)). V Extralige za výhru za riadnu hraciu dobu získa tím 3 body, za výhru po predĺžení 2 body, za prehru po predĺžení 1 bod, a za prehru za riadnu hraciu dobu žiadne body nezíska.